# Map Gaze to video frames

We will download a different neon recording containing a vieo file this time.

In [1]:
import sys
import numpy as np
from pyneon import NeonDataset, NeonRecording
import pandas as pd 

# define path from root directory of repository
recording_dir = '../../data/OpticalFlow/data'

recording = NeonRecording(recording_dir)  

print(recording)


Recording ID: 9265f7c1-e3ce-4108-9dd1-639305591f79
Wearer ID: 990a3de0-5a1a-4f07-bf9b-d82369213dbf
Wearer name: Qian
Recording start time: 2024-07-24 16:31:22.223000
Recording duration: 96.226 s
                 exist              filename                                              path
3d_eye_states     True     3d_eye_states.csv     ..\..\data\OpticalFlow\data\3d_eye_states.csv
blinks            True            blinks.csv            ..\..\data\OpticalFlow\data\blinks.csv
events            True            events.csv            ..\..\data\OpticalFlow\data\events.csv
fixations         True         fixations.csv         ..\..\data\OpticalFlow\data\fixations.csv
gaze              True              gaze.csv              ..\..\data\OpticalFlow\data\gaze.csv
imu               True               imu.csv               ..\..\data\OpticalFlow\data\imu.csv
labels            True            labels.csv            ..\..\data\OpticalFlow\data\labels.csv
saccades          True          saccades.csv

We can see that a scene_video object exists now. The issue with neon recordings is that event data is not naturally synched to the video. Therefore, receiving an overlay of the current gaze or alternatively, the tracking of past fixations, requires further steps.

We will firstly perform a mapping of the current gaze to the video. To do this, we pick the closest timestamp within gaze.csv for every video timestamp in world_timestamps.

In [2]:
recording.map_gaze_to_video("../../data/OpticalFlow/output.pkl")

In calling the method, we can further specify where a pickled output is saved. Why a pickle you may ask...

In [3]:
#load pkl
df = pd.read_pickle("../../data/OpticalFlow/output.pkl")
print(df['fixations'][0])

   fixation_id        x        y status
0            1  765.012  537.417  onset


If we print the output, we can see that our mapped gaze data is a dataframe OF dataframes. For every frame, we have a dataframe of fixations with their corresponding fixation_id, x, y-values and status.

This is motivated by the possibility of multiple fixations being present in any given world frame. Even while we have an active fixation, we can still track past fixations through an optical flow algorithm. Eventually, we have multiple fixations, at ifferent locations and crucially with different status, within the same frame.

The standard choice for an optical flow algorithm is Lucas-Kanade, as we mostly care for the movement of sparse points within the scene. Of course, it would be possible to alternatively compute the entire visual flow field and compute the movement of points accordingly.

Tracking fixations with optical flow takes a bit of time, so make a cup of coffee.

In [4]:
# import warnings and ignore future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

recording.track_fixations_with_optical_flow("../../data/OpticalFlow/output.pkl")


We can now print a single 'fixations'-cell from the outer dictionary:

In [5]:
print(recording.tracked_past_fixations['fixations'][200])

   fixation_id           x           y   status
0           10  827.414000  431.751000   active
1            8  833.097900  462.179871  tracked
2            7  780.811340  415.715637  tracked
3            5  680.597717  408.502167  tracked
4            4  752.542114  389.270447  tracked


As we can see, we now have multiple fixations that are present within the same frame. Fixation 10 is currently active, indicating that Neon interprets it as being maintained. Frames 8, 7, 5, 4 are tracked within the scene, whereas 9 and 6 have already been dropped.

The logic behin the status is the following. Whenever a new frame has been detected, it gets flagged as "onset". For all future frames during which it is monitored, the flag changes to "active". The first frame in which it is no longer active, it gets flagged as "offset", passing it to the optical flow algorithm. While the algorithm manages to track it, the flag changes to "tracked". When the tracking is lost, the flag changes to "lost" after which it is dropped from future tracking.

Calling .overlay_fixations_on_video allows us to create a video with overlaid current and past fixations. The blue dot represents the current gaze location, whereas green dots represent tracked fixations. A red central dot indicates that the current moment is not recognised as a fixation (but rather a saccade or blink)

In [9]:
recording.overlay_fixations_on_video("../../data/OpticalFlow/overlayed.mp4")